In [1]:
import requests
import re
import time
import pymysql

In [22]:
def get_pages(html):
    
    total_pages = re.findall('num.*?(\d+)',html,re.S)
    pages_comt = int(total_pages[2])//10
    pages_truth = int(total_pages[3])//10
    pages_topic = int(total_pages[4])//10
    pages_regulation = int(total_pages[5])//10
    
    print('{}公司包含网民评价：{}条，消费投诉：{}条，新闻报道：{}条，政府监管：{}条'.format(company_name,
            total_pages[2],total_pages[3], total_pages[4], total_pages[5]))
    
    return pages_comt,pages_truth,pages_topic,pages_regulation



def get_url_list(initial_url):
    data_item = ['comt','truth','topic','regulation']
    url_list = []
    url_list.append(initial_url + '?tab=comt')
    url_list.append(initial_url + '?tab=truth')
    url_list.append(initial_url + '?tab=topic')
    url_list.append(initial_url + '?tab=regulation')
    
    return url_list



def get_html(url,headers):
    html = requests.get(url,headers).text
    
    return html


def get_onePage_comt_data(html):
    content_user_name = 'username-click.*?>(.*?)<'
    content_review = 'kb-op-detail.*?pre>(.*?)<'
    content_stars = 'kb-i-small-star-(\d+)'
    content_rate_time = 'class="time">(.*?)<'
    content_praise_num = 'kb-praise-num">(\d+)'
    
    onePage_comt_data = []
    onePage_comt_data.append(re.findall(content_user_name, html,re.S))
    onePage_comt_data.append(re.findall(content_rate_time, html,re.S))
    onePage_comt_data.append(re.findall(content_review, html,re.S))
    onePage_comt_data.append(re.findall(content_stars, html,re.S))
    onePage_comt_data.append(re.findall(content_praise_num, html,re.S))
    
    
    return onePage_comt_data
      
def save_comt_data(cursor,comt_data,conn,company_name):
    
    sql = 'insert into {}(user_name,rate_time,review,stars,praise_num) values(%s,%s,%s,%s,%s)'.format(company_name)
    try:
        cursor.execute(sql,comt_data)
        conn.commit()
        print('-------->数据提交成功<----------')
    except:
        conn.rollback()
        print('-------->数据提交失败，已撤销<----------')

        
# 优化为comt的save函数 
def main(company_name):
    # 初次编写为了方便把创建游标放在main()，后面可以尝试单独写一个函数。
    conn = pymysql.connect(host='localhost',user='root',password='1829200',database='baidu_koubei',port=3306,charset='utf8')
    cursor = conn.cursor()
    # 根据公司名称创建表
    sql = 'create table if not exists {}(user_name varchar(255) not null, \
    rate_time datetime not null, review varchar(255) not null, stars varchar(255) not null,\
    praise_num varchar(255) not null)'.format(company_name)
    cursor.execute(sql)
    print('-------->{}创建成功<----------'.format(company_name))
    
    
    url_list = get_url_list(initial_url)
    html = get_html(url_list[0],headers) #取网民评价页的url来获取页码数量。
    pages = get_pages(html) # 此处输出为四个item的网页数，元组格式。
    
    
    # 先爬取第一个item “网民评价” 从这开始，全部写入save_comt_data()
    url_comt = url_list[0]
    for page in range(pages[0]):
        html = get_html(url_comt + '&' + 'page='+str(page+1), headers)
        onePage_comt_data = get_onePage_comt_data(html)
#         for j in range(10):
#             print(j+1,onePage_comt_data[3][j])

        for i in range(10):
            comt_data = []
            comt_data.append(onePage_comt_data[0][i])
            comt_data.append(onePage_comt_data[1][i])
            comt_data.append(onePage_comt_data[2][i])
            comt_data.append(onePage_comt_data[3][i])
            comt_data.append(onePage_comt_data[4][i])

            save_comt_data(cursor,comt_data,conn,company_name)
            print(pages[0])
            print('第'+str(page+1)+'页', '第'+str(i+1)+'条')
    cursor.close()
    conn.close()

                    
    print('-------->本次爬取数据已全部入库<----------')

In [1]:
company_name = 'shentong'
initial_url = 'https://koubei.baidu.com/s/56987a4b5e0866ac50c2e3e788518db9'

headers = {
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
    'Connection':'keep-alive',
    'Cookie':'BAIDUID=B61886D87A6B85B3031BED011722DFCA:FG=1; BIDUPSID=B61886D87A6B85B3031BED011722DFCA; PSTM=1567062458; BDORZ=FFFB88E999055A3F8A630C64834BD6D0; BDRCVFR[gltLrB7qNCt]=mk3SLVN4HKm; delPer=0; PSINO=2; H_PS_PSSID=1455_21087_29523_29568_29221_22158',
    'Host':'ikefu.baidu.com'
    
}



# main(company_name)

print('修改所爬取的campany_name和initial_url，运行main')


修改所爬取的campany_name和initial_url，运行main
